In [35]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error, r2_score

In [36]:
# Define the dataset class
class EnergyDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X = self.X.iloc[idx]
        y = self.y.iloc[idx]
        return X.values, y.values

In [37]:
# Load the data
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_val = pd.read_csv('X_val.csv')
y_val = pd.read_csv('y_val.csv')
X_test = pd.read_csv('X_test.csv')
y_test = pd.read_csv('y_test.csv')

In [38]:
# Create the dataset
train_dataset = EnergyDataset(X_train, y_train)
val_dataset = EnergyDataset(X_val, y_val)
test_dataset = EnergyDataset(X_test, y_test)

In [39]:
# Data Loader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [40]:
class RNN_QCNN(nn.Module):
    def __init__(self, input_size=24, hidden_size=128, seq_len=10):
        super(RNN_QCNN, self).__init__()

        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)

        # Linear Layer after RNN to reshape data for QCNN
        self.fc_rnn = nn.Linear(hidden_size, 256)

        # QCNN Layers
        self.fc1 = nn.Linear(256, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)

        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)

        self.fc4 = nn.Linear(64, 32)
        self.bn4 = nn.BatchNorm1d(32)

        self.fc5 = nn.Linear(32, 1)

    def forward(self, x):

        # Pass through RNN
        rnn_out, _ = self.rnn(x)

        # Handle case where sequence length might be 1
        if rnn_out.dim() == 3:
            # Take the last output of RNN for each batch (dim=1)
            rnn_out_last = rnn_out[:, -1, :]  # Last time step output
        else:
            rnn_out_last = rnn_out  # If no sequence length, directly use output

        # Pass through Linear layer to reshape data for QCNN
        x = self.fc_rnn(rnn_out_last)

        # QCNN Layers
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu(x)

        x = self.fc3(x)
        x = self.bn3(x)
        x = self.relu(x)

        x = self.fc4(x)
        x = self.bn4(x)
        x = self.relu(x)

        # Output layer
        x = self.fc5(x)
        return x

In [41]:
# Set hyperparameters
learning_rate = 0.001
n_epochs = 200

In [42]:
# Initialize model, optimizer, and loss function
model = RNN_QCNN()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()

In [43]:
# Lists to log training and validation loss and accuracy
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

In [44]:
# Train the model
for epoch in range(n_epochs):
    model.train()  # Set the model to training mode
    epoch_loss = 0
    for batch in train_loader:
        X, y = batch
        X = X.clone().detach().float()
        y = y.clone().detach().float()  
        optimizer.zero_grad()
        outputs = model(X)
        loss = loss_fn(outputs, y)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()

    # Calculate average loss for the epoch
    average_loss = epoch_loss / len(train_loader)
    train_losses.append(average_loss)

    # Calculate training accuracy (R² score)
    with torch.no_grad():
        y_train_pred = model(torch.tensor(X_train.values, dtype=torch.float32)).flatten().numpy()
        r2 = r2_score(y_train.values, y_train_pred)
        accuracy = r2 * 100
        train_accuracies.append(accuracy)

    # Validate the model
    model.eval()  # Set the model to evaluation mode
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            X, y = batch
            X = X.clone().detach().float()
            y = y.clone().detach().float()  
            outputs = model(X)
            loss = loss_fn(outputs, y)
            val_loss += loss.item()
        
        average_val_loss = val_loss / len(val_loader)
        val_losses.append(average_val_loss)

        y_val_pred = model(torch.tensor(X_val.values, dtype=torch.float32)).flatten().numpy()
        val_r2 = r2_score(y_val.values, y_val_pred)
        val_accuracy = val_r2 * 100
        val_accuracies.append(val_accuracy)

    print(f'Epoch [{epoch + 1}/{n_epochs}], Train Loss: {average_loss}')

Epoch [1/200], Train Loss: 0.007794812729799539
Epoch [2/200], Train Loss: 0.002509972009631153
Epoch [3/200], Train Loss: 0.002340654270458454
Epoch [4/200], Train Loss: 0.002024359110897016
Epoch [5/200], Train Loss: 0.002106437262463671
Epoch [6/200], Train Loss: 0.0020555776480243528
Epoch [7/200], Train Loss: 0.002034321620940769
Epoch [8/200], Train Loss: 0.00187443094997396
Epoch [9/200], Train Loss: 0.002017519344494349
Epoch [10/200], Train Loss: 0.0018684194105506706
Epoch [11/200], Train Loss: 0.0018874592719371327
Epoch [12/200], Train Loss: 0.0018324625962644553
Epoch [13/200], Train Loss: 0.001799427328183227
Epoch [14/200], Train Loss: 0.0018228527955420077
Epoch [15/200], Train Loss: 0.001691327332144098
Epoch [16/200], Train Loss: 0.0016240321768045843
Epoch [17/200], Train Loss: 0.0016136439667402667
Epoch [18/200], Train Loss: 0.0015940944676095016
Epoch [19/200], Train Loss: 0.001571750298582292
Epoch [20/200], Train Loss: 0.0014987360756489724
Epoch [21/200], Train

In [45]:
# Save trained model and metrics
torch.save({
    'model_state_dict': model.state_dict(),
    'train_losses': train_losses,
    'train_accuracies': train_accuracies,
    'val_losses': val_losses,
    'val_accuracies': val_accuracies
}, 'models/qcnn_rnn_model.pth')

In [46]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Test the model on the test set
model.eval()  # Set the model to evaluation mode
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

with torch.no_grad():
    y_test_pred = model(X_test_tensor).flatten().numpy()
    
    # Calculate MSE, MAE, RMSE, R² score, and MAPE
    test_mse = mean_squared_error(y_test.values, y_test_pred)
    test_r2 = r2_score(y_test.values, y_test_pred)
    test_mae = mean_absolute_error(y_test.values, y_test_pred)
    test_rmse = mean_squared_error(y_test.values, y_test_pred, squared=False)
    test_mape = np.mean(np.abs((y_test.values - y_test_pred) / y_test.values)) * 100

    # Calculate accuracy as R² score in percentage
    test_accuracy = test_r2 * 100
    
    print(f'Mean Squared Error on test data: {test_mse}')
    print(f'R² score (Accuracy) on test data: {test_r2}')
    print(f'Accuracy: {test_accuracy}%')
    print(f'Mean Absolute Error (MAE) on test data: {test_mae}')
    print(f'Root Mean Squared Error (RMSE) on test data: {test_rmse}')
    print(f'Mean Absolute Percentage Error (MAPE) on test data: {test_mape}%')


c:\Users\Asus\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Mean Squared Error on test data: 0.0003835035616209055
R² score (Accuracy) on test data: 0.9902209995058393
Accuracy: 99.02209995058394%
Mean Absolute Error (MAE) on test data: 0.01363999795374514
Root Mean Squared Error (RMSE) on test data: 0.019583246963180173
Mean Absolute Percentage Error (MAPE) on test data: 87.99191732960747%
